# NYC 311 Calls Analysis
NYC 311 is a non-emergency phone number and online system that New York City residents, visitors, and businesses can use to report various issues, obtain information, and access non-emergency services provided by the city. The service is named after its phone number, 311

This project starts with a large database of complaints filed by residents of New York City via 311 calls. The full dataset is available at the [NYC open data portal](https://data.cityofnewyork.us/Social-Services/311-Service-Requests-from-2010-to-Present/erm2-nwe9). The project utilized the subset which is about 6 GB and 10 million complaints, so we can infer that NYC residents have a lot to complain about. The notebook employs a combination of`sqlite`, `pandas`, and `bokeh`to analyze and visualize the data.

In [79]:
import sys
print(sys.version)

from IPython.display import display
import pandas as pd

import sqlite3 as db
import urllib.parse
import requests
import os
import hashlib
import io

3.10.8 (tags/v3.10.8:aaaf517, Oct 11 2022, 16:50:30) [MSC v.1933 64 bit (AMD64)]


## Helper Functions

Function `canonicalize_tibble(X)`, that, given a tibble `X`, returns a new copy `Y` of `X` in _canonical order_. `Y` is in canonical order if it has the following properties.

1. The variables appear in sorted order by name, ascending from left to right.
2. The rows appear in lexicographically sorted order by variable, ascending from top to bottom.
3. The row labels (`Y.index`) go from 0 to `n-1`, where `n` is the number of observations.

For instance, here is a **non-canonical tibble** ...

|   |  c  | a | b |
|:-:|:---:|:-:|:-:|
| 2 | hat | x | 1 |
| 0 | rat | y | 4 |
| 3 | cat | x | 2 |
| 1 | bat | x | 2 |


... and here is its **canonical counterpart.**

|   | a | b |  c  |
|:-:|:-:|:-:|:---:|
| 0 | x | 1 | hat |
| 1 | x | 2 | bat |
| 2 | x | 2 | cat |
| 3 | y | 4 | rat |


In [80]:
def canonicalize_tibble(X):
    # Enforce Property 1:
    var_names = sorted(X.columns)
    Y = X[var_names].copy()
    Y = Y.sort_values(by = var_names)   #['a', 'b', 'c'])
    Y = Y.reset_index(drop = True)
    return Y

Function `tibbles_are_equivalent(A, B)` to determine if two tibbles, `A` and `B`, are equivalent. "Equivalent" means that `A` and `B` have identical variables and observations, up to permutations. If `A` and `B` are equivalent, then the function should return `True`. Otherwise, it should return `False`.

The last condition, "up to permutations," means that the variables and observations might not appear in the table in the same order. For example, the following two tibbles are equivalent:


| a | b |  c  |
|:-:|:-:|:---:|
| x | 1 | hat |
| y | 2 | cat |
| z | 3 | bat |
| w | 4 | rat |

| b |  c  | a |
|:-:|:---:|:-:|
| 2 | cat | y |
| 3 | bat | z |
| 1 | hat | x |
| 4 | rat | w |

By contrast, the following table would not be equivalent to either of the above tibbles.

| a | b |  c  |
|:-:|:-:|:---:|
| 2 | y | cat |
| 3 | z | bat |
| 1 | x | hat |
| 4 | w | rat |

> **Note**: Unlike Pandas data frames, tibbles conceptually do not have row labels. So we should ignore row labels.

In [10]:
def tibbles_are_equivalent(A, B):
    """Given two tidy tables ('tibbles'), returns True iff they are
    equivalent.
    """
    for col in A.columns:
        if sorted(A[col]) == sorted(B[col]):
            continue
        else:
            return False
    return True

Function to cast a data frame into a tibble, given a key column containing new variable names and a value column containing the corresponding cells.

- verifies that the given `key` and `value` columns are actual columns of the input data frame;
- computes the list of columns, `fixed_vars`, that should remain unchanged; and
- initializes and empty tibble.


In [67]:
def cast(df, key, value, join_how = 'outer'):
    """Casts the input data frame into a tibble,
    given the key column and value column.
    """
    assert type(df) is pd.DataFrame
    assert key in df.columns and value in df.columns
    assert join_how in ['outer', 'inner']
    
    fixed_vars = df.columns.difference([key, value])
    tibble = pd.DataFrame(columns=fixed_vars) # empty frame
    
    # Use the input DataFrame df instead of an undefined variable table2
    tibble = df.pivot_table(values=value, index=list(fixed_vars), columns=key).reset_index()
    
    return tibble

## Data setup

This is small subset (about 250+ MiB) of the full data as of 2015.

> Download this file manually from the following link and place it locally in a (nested) subdirectory or folder named `resource/asnlib/publicdata`.
>
> [Link to the pre-constructed NYC 311 Database on MS OneDrive](https://onedrive.live.com/download?cid=FD520DDC6BE92730&resid=FD520DDC6BE92730%21616&authkey=AEeP_4E1uh-vyDE)

**Connecting.** Open up a connection to this dataset.

In [14]:
DB_FILENAME = "resource/asnlib/publicdata/NYC-311-2M.db"
# Construct the URI for connecting to the SQLite database file
db_uri = 'file:{}?mode=ro'.format(urllib.parse.quote_plus(DB_FILENAME))

# Connect to the SQLite database
disk_engine = db.connect(db_uri, uri=True)

**Preview the data.** This sample database has just a single table, named `data`. Let's query it and see how long it takes to read. To carry out the query, we will use the SQL reader built into `pandas`.

In [15]:
import time

print ("Reading ...")
start_time = time.time ()

# Perform SQL query through the disk_engine connection.
# The return value is a pandas data frame.
df = pd.read_sql_query ('select * from data', disk_engine)

elapsed_time = time.time () - start_time
print ("==> Took %g seconds." % elapsed_time)

# Dump the first few rows
df.head()

Reading ...
==> Took 23.1888 seconds.


,index,CreatedDate,ClosedDate,Agency,ComplaintType,Descriptor,City
0,1,2015-09-15 02:14:04.000000,None,NYPD,Illegal Parking,Blocked Hydrant,None
1,2,2015-09-15 02:12:49.000000,None,NYPD,Noise - Street/Sidewalk,Loud Talking,NEW YORK
2,3,2015-09-15 02:11:19.000000,None,NYPD,Noise - Street/Sidewalk,Loud Talking,NEW YORK
3,4,2015-09-15 02:09:46.000000,None,NYPD,Noise - Commercial,Loud Talking,BRONX
4,5,2015-09-15 02:08:01.000000,2015-09-15 02:08:18.000000,DHS,Homeless Person Assistance,Status Call,NEW YORK


**Partial queries: `LIMIT` clause.** The preceding command was overkill for what we wanted, which was just to preview the table. Instead, we could have used the `LIMIT` option to ask for just a few results. This SQL query selects all columns (*) from the table named 'data' and limits the result to the first 5 rows (LIMIT 5).

In [16]:
query = '''
  SELECT *
    FROM data
    LIMIT 5
'''
start_time = time.time ()
df = pd.read_sql_query (query, disk_engine)
elapsed_time = time.time () - start_time
print ("==> LIMIT version took %g seconds." % elapsed_time)

df

==> LIMIT version took 0.0338626 seconds.


,index,CreatedDate,ClosedDate,Agency,ComplaintType,Descriptor,City
0,1,2015-09-15 02:14:04.000000,None,NYPD,Illegal Parking,Blocked Hydrant,None
1,2,2015-09-15 02:12:49.000000,None,NYPD,Noise - Street/Sidewalk,Loud Talking,NEW YORK
2,3,2015-09-15 02:11:19.000000,None,NYPD,Noise - Street/Sidewalk,Loud Talking,NEW YORK
3,4,2015-09-15 02:09:46.000000,None,NYPD,Noise - Commercial,Loud Talking,BRONX
4,5,2015-09-15 02:08:01.000000,2015-09-15 02:08:18.000000,DHS,Homeless Person Assistance,Status Call,NEW YORK


**Finding unique values: `DISTINCT` qualifier.** This SQL query retrieves distinct values from the 'City' column in the 'data' table.

In [21]:
query = 'SELECT DISTINCT City FROM data'
df = pd.read_sql_query(query, disk_engine)

print("Found {} unique cities. The first few are:".format(len(df)))
df.head()

Found 547 unique cities. The first few are:


,City
0,None
1,NEW YORK
2,BRONX
3,STATEN ISLAND
4,ELMHURST


**Grouping Information: GROUP BY operator.** The GROUP BY operator lets group information using a particular column or multiple columns of the table. The output generated is more of a pivot table. This SQL query selects distinct combinations of ComplaintType, Descriptor, and Agency from the 'data' table, grouping the result by ComplaintType.

In [22]:
query = '''
  SELECT ComplaintType, Descriptor, Agency
    FROM data
    GROUP BY ComplaintType
'''

df = pd.read_sql_query(query, disk_engine)
print(df.shape)
df.head()

(200, 3)


,ComplaintType,Descriptor,Agency
0,AGENCY,HOUSING QUALITY STANDARDS,HPD
1,APPLIANCE,ELECTRIC/GAS RANGE,HPD
2,Adopt-A-Basket,10A Adopt-A-Basket,DSNY
3,Agency Issues,Bike Share,DOT
4,Air Quality,"Air: Odor/Fumes, Vehicle Idling (AD3)",DEP


**`GROUP BY` aggregations.** A common pattern is to combine grouping with aggregation. For example, suppose we want to count how many times each complaint occurs. This SQL query counts the occurrences of each unique 'ComplaintType'.

In [23]:
query = '''
  SELECT ComplaintType, COUNT(*)
    FROM data
    GROUP BY ComplaintType
    LIMIT 10
'''

df = pd.read_sql_query(query, disk_engine)
df.head()

,ComplaintType,COUNT(*)
0,AGENCY,2
1,APPLIANCE,11263
2,Adopt-A-Basket,50
3,Agency Issues,7428
4,Air Quality,8151


**Character-case conversions.** From the two preceding examples, observe that the strings employ a mix of case conventions (i.e., lowercase vs. uppercase vs. mixed case). A convenient way to query and "normalize" case is to apply SQL's `UPPER()` and `LOWER()` functions. This SQL query converts the 'ComplaintType,' 'Descriptor,' and 'Agency' columns to lowercase using the LOWER function.

In [24]:
query = '''
  SELECT LOWER(ComplaintType), LOWER(Descriptor), LOWER(Agency)
    FROM data
    GROUP BY LOWER(ComplaintType)
    LIMIT 10
'''

df = pd.read_sql_query(query, disk_engine)
df.head()

,LOWER(ComplaintType),LOWER(Descriptor),LOWER(Agency)
0,adopt-a-basket,10a adopt-a-basket,dsny
1,agency,housing quality standards,hpd
2,agency issues,bike share,dot
3,air quality,"air: odor/fumes, vehicle idling (ad3)",dep
4,animal abuse,other (complaint details),nypd


**Filtered aggregations: `HAVING` clauses.** A common pattern for aggregation queries (e.g., `GROUP BY` plus `COUNT()`) is to filter the grouped results. We cannot do that with a `WHERE` clause alone, because `WHERE` is applied *before* grouping.

As an example, recall that some `ComplaintType` values are in all uppercase whereas some use mixed case. Since we didn't inspect all of them, there might even be some are all lowercase. Worse, we would expect some inconsistencies. For instance, it turns out that both `"Plumbing"` (mixed case) and `"PLUMBING"` (all caps) appear. Here is a pair of queries that makes this point.

In [81]:
query0 = "SELECT DISTINCT ComplaintType FROM data"
df0 = pd.read_sql_query(query0, disk_engine)
print("Found {} unique `ComplaintType` strings.".format(len(df0)))
display(df0.head())

query1 = "SELECT DISTINCT LOWER(ComplaintType) FROM data"
df1 = pd.read_sql_query(query1, disk_engine)
print("\nFound {} unique `LOWER(ComplaintType)` strings.".format(len(df1)))
display(df1.head())

print("\n==> Therefore, there are {} cases that are duplicated.".format(len(df0) - len(df1)))

Found 200 unique `ComplaintType` strings.


,ComplaintType
0,Illegal Parking
1,Noise - Street/Sidewalk
2,Noise - Commercial
3,Homeless Person Assistance
4,Highway Condition



Found 198 unique `LOWER(ComplaintType)` strings.


,LOWER(ComplaintType)
0,illegal parking
1,noise - street/sidewalk
2,noise - commercial
3,homeless person assistance
4,highway condition



==> Therefore, there are 2 cases that are duplicated.


We wanted a query that identifies these inconsistent capitalizations. Here is one way to do it, which demonstrates the `HAVING` clause. (It also uses a **nested query**, that is, it performs one query and then selects immediately from that result.)

In [26]:
query2 = '''
    SELECT ComplaintType, COUNT(*)
      FROM (SELECT DISTINCT ComplaintType FROM data)
      GROUP BY LOWER(ComplaintType)
      HAVING COUNT(*) >= 2
'''
df2 = pd.read_sql_query(query2, disk_engine)
df2

,ComplaintType,COUNT(*)
0,Elevator,2
1,PLUMBING,2


**Set membership: `IN` operator.** Another common idiom is to ask for rows whose attributes fall within a set, for which we can use the `IN` operator. Let's use it to see the two inconsistent-capitalization complaint types from above. This SQL query retrieves distinct values from the 'ComplaintType' column where the lowercase 'ComplaintType' is either "plumbing" or "elevator." 

In [27]:
query = '''
    SELECT DISTINCT ComplaintType
      FROM data
      WHERE LOWER(ComplaintType) IN ("plumbing", "elevator")
'''
df = pd.read_sql_query(query, disk_engine)
df.head()

,ComplaintType
0,PLUMBING
1,Elevator
2,Plumbing
3,ELEVATOR


**Renaming columns: `AS` operator.** We want to  rename a result column. The following query counts the number of complaints by "Agency," using the `COUNT(*)` function and `GROUP BY` clause. If we wish to refer to the counts column of the resulting data frame, we can give it a more "friendly" name using the `AS` operator.

In [28]:
query = '''
  SELECT Agency, COUNT(*) AS NumComplaints
    FROM data
    GROUP BY Agency
'''
df = pd.read_sql_query(query, disk_engine)
df.head()

,Agency,NumComplaints
0,3-1-1,1289
1,ACS,3
2,AJC,6
3,CAU,1
4,CCRB,1


**Ordering results: `ORDER BY` clause.** We want to execute the previous query by number of complaints.

In [29]:
query = '''
  SELECT Agency, COUNT(*) AS NumComplaints
    FROM data
    GROUP BY UPPER(Agency)
    ORDER BY NumComplaints
'''
df = pd.read_sql_query(query, disk_engine)
df.tail()

,Agency,NumComplaints
45,DSNY,152004
46,DEP,181121
47,DOT,322969
48,NYPD,340694
49,HPD,640096


Note that the above example prints the bottom (tail) of the data frame. We could have also asked for the query results in reverse (descending) order, by prefixing the `ORDER BY` attribute with a `-` (minus) symbol. Alternatively, we can use `DESC` to achieve the same result.

In [30]:
query = '''
  SELECT Agency, COUNT(*) AS NumComplaints
    FROM data
    GROUP BY UPPER(Agency)
    ORDER BY -NumComplaints
'''

# Alternative: query =
'''
SELECT Agency, COUNT(*) AS NumComplaints 
    FROM data 
    GROUP BY UPPER(Agency)
    ORDER BY NumComplaints DESC 
'''

df = pd.read_sql_query(query, disk_engine)
df.head()

,Agency,NumComplaints
0,HPD,640096
1,NYPD,340694
2,DOT,322969
3,DEP,181121
4,DSNY,152004


Function 'make_barchart' is designed to create a basic bar chart using the Bokeh library based on the input DataFrame (df). The function takes three parameters: df (the DataFrame containing the data), labels (the column name representing the categorical labels for the x-axis), and values (the column name representing the numerical values for the y-axis). Additionally, we can provide optional keyword arguments (kwargs_figure) to customize the appearance of the Bokeh figure.

Inside the function, it first checks if the input DataFrame is an instance of the Pandas DataFrame class and ensures that the labels and values parameters are strings. Then, it uses Bokeh's figure function to create a basic bar chart, specifying the unique values of the labels column for the x-axis categories. The heights of the bars are determined by the corresponding values in the values column. The width of the bars is set to 0.9 for spacing. The resulting Bokeh plot (p) is returned by the function. This function provides a simple way to generate bar charts from Pandas DataFrames using Bokeh.

In [39]:
from bokeh.plotting import figure

def make_barchart(df, labels, values, kwargs_figure={}):
    from pandas import DataFrame
    assert isinstance(df, DataFrame)
    assert type(labels) is str and type(values) is str
    p = figure(x_range=list(df[labels].unique()), **kwargs_figure)
    p.vbar(x=list(df[labels]), top=list(df[values]), width=0.9)
    return p

In [45]:
from bokeh.io import output_notebook, show
output_notebook()

p = make_barchart(df[:20], 'Agency', 'NumComplaints',
                  {'title': 'Top 20 agencies by number of complaints',
                   'width': 800, 'height': 320})
p.xaxis.major_label_orientation = 0.66
show(p, notebook_handle=True)

Loading BokehJS ...

> What is the most common type of complaint? What, if anything, does it tell about NYC?

In [87]:
del query # clears any existing `query` variable; we should define it, below!
query = '''Select LOWER(ComplaintType) as type, Count(ComplaintType) as freq
from data group by type ORDER BY -freq '''
# Runs the `query`:
df_complaint_freq = pd.read_sql_query(query, disk_engine)
df_complaint_freq.head()

,type,freq
0,heat/hot water,241430
1,street condition,124347
2,street light condition,98577
3,blocked driveway,95080
4,illegal parking,83961


Let's also visualize the result, as a bar chart showing complaint types on the x-axis and the number of complaints on the y-axis.

In [48]:
p = make_barchart(df_complaint_freq[:25], 'type', 'freq',
                  {'title': 'Top 25 complaints by type',
                   'width': 800, 'height': 320})
p.xaxis.major_label_orientation = 0.66
show(p, notebook_handle=True)

The prevalence of "heat/hot water" complaints, with a frequency of 241,430, becomes especially significant when considering that New York City is densely populated. The high population density may contribute to increased demand for essential services like heating and hot water, and it emphasizes the challenges that urban areas face in meeting the needs of a large and concentrated population. The data suggests that ensuring adequate heating and hot water services is crucial in a crowded city like New York, where the sheer number of residents amplifies the impact of such issues. Addressing these concerns becomes essential for maintaining the quality of life and well-being of the city's residents in the context of its urban density.

**Simple substring matching: the `LIKE` operator.** Suppose we just want to look at the counts for all complaints that have the word `noise` in them. We can use the `LIKE` operator combined with the string wildcard, `%`, to look for case-insensitive substring matches.

In [49]:
query = '''
  SELECT LOWER(ComplaintType) AS type, COUNT(*) AS freq
    FROM data
    WHERE LOWER(ComplaintType) LIKE '%noise%'
    GROUP BY type
    ORDER BY -freq
'''

df_noisy = pd.read_sql_query(query, disk_engine)
print("Found {} queries with 'noise' in them.".format(len(df_noisy)))
df_noisy

Found 8 queries with 'noise' in them.


,type,freq
0,noise,54165
1,noise - street/sidewalk,48436
2,noise - commercial,42422
3,noise - vehicle,18370
4,noise - park,4020
5,noise - helicopter,1715
6,noise - house of worship,1143
7,collection truck noise,184


Query that contains an SQL query that will return the top 10 cities with the largest number of complaints, in descending order. It should return a table with two columns, one named `name` holding the name of the city, and one named `freq` holding the number of complaints by that city. 

Like complaint types, cities are not capitalized consistently. Therefore, standardize the city names by converting them to **uppercase**.

In [85]:
del query # define a new `query` variable, below
query = '''SELECT City AS name, count(*) AS freq
            FROM data 
            GROUP BY name
            ORDER BY -freq
'''
# Runs `query`:
df_whiny_cities = pd.read_sql_query(query, disk_engine)
df_whiny_cities

,name,freq
0,BROOKLYN,579363
1,NEW YORK,385655
2,BRONX,342533
3,None,168692
4,STATEN ISLAND,92509
...,...,...
542,WOODCLIFF LAKE,1
543,WOODNERE,1
544,WOODSTOCK,1
545,WOOSIDE,1


**Case-insensitive grouping: `COLLATE NOCASE`.** Another way to carry out the preceding query in a case-insensitive way is to add a `COLLATE NOCASE` qualifier to the `GROUP BY` clause.

The next example demonstrates this clause. Note that it also filters out the 'None' cases, where the `<>` operator denotes "not equal to." Lastly, this query ensures that the returned city names are uppercase.

> The `COLLATE NOCASE` clause modifies the column next to which it appears. So if we are grouping by more than one key and want to be case-insensitive, we need to write, `... GROUP BY ColumnA COLLATE NOCASE, ColumnB COLLATE NOCASE ...`.

In [51]:
query = '''
  SELECT UPPER(City) AS name, COUNT(*) AS freq
    FROM data
    WHERE name <> 'None'
    GROUP BY City COLLATE NOCASE
    ORDER BY -freq
    LIMIT 10
'''
df_whiny_cities2 = pd.read_sql_query(query, disk_engine)
df_whiny_cities2

,name,freq
0,BROOKLYN,579363
1,NEW YORK,385655
2,BRONX,342533
3,STATEN ISLAND,92509
4,JAMAICA,46683
5,FLUSHING,35504
6,ASTORIA,31873
7,RIDGEWOOD,21618
8,WOODSIDE,15932
9,CORONA,15740


Lastly, for later use, let's save the names of just the top seven (7) cities by numbers of complaints.

In [52]:
TOP_CITIES = list(df_whiny_cities2.head(7)['name'])
TOP_CITIES

['BROOKLYN',
 'NEW YORK',
 'BRONX',
 'STATEN ISLAND',
 'JAMAICA',
 'FLUSHING',
 'ASTORIA']

Implement a function that takes a list of strings, `str_list`, and returns a single string consisting of each value, `str_list[i]`, enclosed by double-quotes and separated by a comma-space delimiters. For example, if

```python
   assert str_list == ['a', 'b', 'c', 'd']
```

then

```python
   assert strs_to_args(str_list) == '"a", "b", "c", "d"'
```

In [82]:
def strs_to_args(str_list):
    assert type (str_list) is list
    assert all ([type (s) is str for s in str_list])
    str_convert = ''
    for item in str_list:
        if item == str_list[len(str_list)-1]:
            str_convert += '"'+item+'"'
        else:
            str_convert += '"'+item+'"'+', '
    return str_convert

Suppose we want to look at the number of complaints by type _and_ by city **for only the top cities**, i.e., those in the list `TOP_CITIES` computed above. We will execute an SQL query to produce a tibble named `df_complaints_by_city` with the variables {`complaint_type`, `city_name`, `complaint_count`}.

Output converts all city names to uppercase and convert all complaint types to lowercase.

In [86]:
query = '''
SELECT LOWER(ComplaintType) AS complaint_type, UPPER(City) AS city_name, COUNT(*) AS complaint_count
FROM data
WHERE city_name IN ({})
GROUP BY city_name, complaint_type
ORDER BY city_name, complaint_count DESC
'''.format(", ".join(["'{}'".format(city.upper()) for city in TOP_CITIES]))
df_complaints_by_city = pd.read_sql_query(query, disk_engine)
df_complaints_by_city = canonicalize_tibble(df_complaints_by_city)
# Previews the results of the query:
print("Found {} records.".format(len(df_complaints_by_city)))
display(df_complaints_by_city.head(10))

Found 1042 records.


,city_name,complaint_count,complaint_type
0,ASTORIA,1,bottled water
1,ASTORIA,1,bridge condition
2,ASTORIA,1,city vehicle placard complaint
3,ASTORIA,1,open flame permit
4,ASTORIA,1,panhandling
5,ASTORIA,1,stalled sites
6,ASTORIA,1,window guard
7,ASTORIA,2,beach/pool/sauna complaint
8,ASTORIA,2,cranes and derricks
9,ASTORIA,2,drinking water


Let's use Bokeh to visualize the results as a stacked bar chart.

In [55]:
# Let's consider only the top 25 complaints (by total)
top_complaints = df_complaint_freq[:25]
print("Top complaints:")
display(top_complaints)

Top complaints:


,type,freq
0,heat/hot water,241430
1,street condition,124347
2,street light condition,98577
3,blocked driveway,95080
4,illegal parking,83961
5,unsanitary condition,81394
6,paint/plaster,69929
7,water system,69209
8,plumbing,60105
9,noise,54165


In [56]:
# Plot subset of data corresponding to the top complaints
df_plot = top_complaints.merge(df_complaints_by_city,
                               left_on=['type'],
                               right_on=['complaint_type'],
                               how='left')
df_plot.dropna(inplace=True)
print("Data to plot (first few rows):")
display(df_plot.head())
print("...")

Data to plot (first few rows):


,type,freq,city_name,complaint_count,complaint_type
0,heat/hot water,241430,ASTORIA,3396.0,heat/hot water
1,heat/hot water,241430,BRONX,79690.0,heat/hot water
2,heat/hot water,241430,BROOKLYN,72410.0,heat/hot water
3,heat/hot water,241430,FLUSHING,2741.0,heat/hot water
4,heat/hot water,241430,JAMAICA,3376.0,heat/hot water


...


The 'make_stacked_barchart' function is designed to create a stacked bar chart using the Bokeh library in Python. It takes as input a DataFrame (df) and three variable names (x_var, cat_var, and y_var), where x_var represents the x-axis variable, cat_var represents the categorical variable for stacking, and y_var is the y-axis variable indicating the values to be plotted. The function also provides optional parameters for customizing the chart appearance, such as x_labels for specifying x-axis labels, bar_labels for legend labels, and kwargs_figure for additional figure settings.

The core of the function involves pivoting the DataFrame to organize the data for plotting, handling missing values if specified (fillna), and cumulatively summing the values along the specified categorical variable. It utilizes the Bokeh library to generate a stacked bar chart, with each bar representing a category, and the height of each segment within a bar corresponding to the cumulative sum of values. The resulting chart effectively visualizes the distribution of values across different categories and provides insight into the data's composition.

In [60]:
def make_stacked_barchart(df, x_var, cat_var, y_var, fillna=True,
                          x_labels=None, bar_labels=None,
                          kwargs_figure={}):
    from bokeh.models import ColumnDataSource
    from bokeh.plotting import figure
    from bokeh.core.properties import value
    from bokeh.models.ranges import FactorRange
    from bokeh.io import show
    from bokeh.plotting import figure

    assert type(x_var) is str, "x-variable should be a string but isn't."
    assert type(cat_var) is str, "category variable should be a string but isn't."
    assert type(y_var) is str, "y-variable should be a string but isn't."
    
    pt = df.pivot(x_var, cat_var, y_var)
    if fillna:
        pt.fillna(0, inplace=True)
    pt = pt.cumsum(axis=1)
    bar_vars = pt.columns
    
    from bokeh.palettes import brewer
    assert len(bar_vars) in brewer['Dark2'], "Not enough colors."

    x = x_labels if x_labels is not None else FactorRange(factors=list(pt.index))
    legend = bar_labels if bar_labels is not None else list(bar_vars)
    colors = brewer['Dark2'][len(bar_vars)]
    source = ColumnDataSource(data=df)

    p = figure(x_range=x, **kwargs_figure)
    bot = 0
    for k, var in enumerate(bar_vars):
        p.vbar(x=pt.index, bottom=bot, top=pt[var], color=colors[k], legend_label=var, width=0.25)
        bot = pt[var]
    return p

In [61]:
# Some code to render a Bokeh stacked bar chart
kwargs_figure = {'title': "Distribution of the top 25 complaints among top 7 cities with the most complaints",
                 'width': 800,
                 'height': 400,
                 'tools': "hover,crosshair,pan,box_zoom,wheel_zoom,save,reset,help"}

def plot_complaints_stacked_by_city(df, y='complaint_count'):
    p = make_stacked_barchart(df, 'complaint_type', 'city_name', y,
                              x_labels=list(top_complaints['type']), bar_labels=TOP_CITIES,
                              kwargs_figure=kwargs_figure)
    p.xaxis.major_label_orientation = 0.66
    from bokeh.models import HoverTool
    hover_tool = p.select(dict(type=HoverTool))
    hover_tool.tooltips = [("y", "$y{int}")]
    return p

show(plot_complaints_stacked_by_city(df_plot))

C:\Users\billy\AppData\Local\Temp\ipykernel_9896\1837551374.py:15: FutureWarning: In a future version of pandas all arguments of DataFrame.pivot will be keyword-only.
  pt = df.pivot(x_var, cat_var, y_var)


Suppose we want to create a different stacked bar plot that shows, for each complaint type $t$ and city $c$, the fraction of all complaints of type $t$ (across all cities, not just the top ones) that occurred in city $c$. Store result in a dataframe named `df_plot_fraction`. It should have the same columns as `df_plot`, **except** that the `complaint_count` column should be replaced by one named `complaint_frac`, which holds the fractional values.

> **Note.** The test cell will create the chart in addition to checking the result. Note that the normalized bars will not necessarily add up to 1.

In [62]:
df_plot['complaint_frac'] = df_plot['complaint_count'] / df_plot['freq']
df_plot_fraction = df_plot.drop('complaint_count', axis=1)
df_plot_fraction.head()

,type,freq,city_name,complaint_type,complaint_frac
0,heat/hot water,241430,ASTORIA,heat/hot water,0.014066
1,heat/hot water,241430,BRONX,heat/hot water,0.330075
2,heat/hot water,241430,BROOKLYN,heat/hot water,0.299921
3,heat/hot water,241430,FLUSHING,heat/hot water,0.011353
4,heat/hot water,241430,JAMAICA,heat/hot water,0.013983


In [68]:
df_plot_stacked_fraction = cast(df_plot_fraction, key='city_name', value='complaint_frac')
show(plot_complaints_stacked_by_city(df_plot_fraction, y='complaint_frac'))


C:\Users\billy\AppData\Local\Temp\ipykernel_9896\1837551374.py:15: FutureWarning: In a future version of pandas all arguments of DataFrame.pivot will be keyword-only.
  pt = df.pivot(x_var, cat_var, y_var)


## Dates and times in SQL

Recall that the input data had a column with timestamps corresponding to when someone submitted a complaint. Let's quickly summarize some of the features in SQL and Python for reasoning about these timestamps.

The `CreatedDate` column is actually a specially formatted date and time stamp, where we can query against by comparing to strings of the form, `YYYY-MM-DD hh:mm:ss`.

For example, let's look for all complaints on September 15, 2015.

In [69]:
query = '''
  SELECT LOWER(ComplaintType), CreatedDate, UPPER(City)
    from data
    where CreatedDate >= "2015-09-15 00:00:00.0"
      and CreatedDate < "2015-09-16 00:00:00.0"
    order by CreatedDate
'''
df = pd.read_sql_query (query, disk_engine)
df

,LOWER(ComplaintType),CreatedDate,UPPER(City)
0,illegal parking,2015-09-15 00:01:23.000000,None
1,blocked driveway,2015-09-15 00:02:29.000000,REGO PARK
2,taxi complaint,2015-09-15 00:02:34.000000,NEW YORK
3,opinion for the mayor,2015-09-15 00:03:07.000000,None
4,opinion for the mayor,2015-09-15 00:03:07.000000,None
...,...,...,...
113,homeless person assistance,2015-09-15 02:08:01.000000,NEW YORK
114,noise - commercial,2015-09-15 02:09:46.000000,BRONX
115,noise - street/sidewalk,2015-09-15 02:11:19.000000,NEW YORK
116,noise - street/sidewalk,2015-09-15 02:12:49.000000,NEW YORK


This next example shows how to extract just the hour from the time stamp, using SQL's `strftime()`.

In [70]:
query = '''
  SELECT CreatedDate, STRFTIME('%H', CreatedDate) AS Hour, LOWER(ComplaintType)
    FROM data
    LIMIT 5
'''
df = pd.read_sql_query (query, disk_engine)
df

,CreatedDate,Hour,LOWER(ComplaintType)
0,2015-09-15 02:14:04.000000,02,illegal parking
1,2015-09-15 02:12:49.000000,02,noise - street/sidewalk
2,2015-09-15 02:11:19.000000,02,noise - street/sidewalk
3,2015-09-15 02:09:46.000000,02,noise - commercial
4,2015-09-15 02:08:01.000000,02,homeless person assistance


We construct a tibble called `df_complaints_by_hour`, which contains the total number of complaints during a given hour of the day. That is, the variables or column names should be {`hour`, `count`} where each observation is the total number of complaints (`count`) that occurred during a given `hour`.

> Interpret `hour` as follows: when `hour` is `02`, that corresponds to the open time interval [`02:00:00`, `03:00:00.0`).

In [83]:
query = '''SELECT strftime('%H', CreatedDate) as hour, count(*) as count
            FROM data GROUP BY hour'''
df_complaints_by_hour = pd.read_sql_query(query, disk_engine)
# Displays answer:
display(df_complaints_by_hour)

,hour,count
0,00,564703
1,01,23489
2,02,15226
3,03,10164
4,04,8692
5,05,10224
6,06,23051
7,07,42273
8,08,73811
9,09,100077


Let's take a quick look at the hour-by-hour breakdown above.

In [73]:
p = make_barchart(df_complaints_by_hour, 'hour', 'count',
                  {'title': 'Complaints by hour',
                   'width': 800, 'height': 320})
show(p, notebook_handle=True)

An unusual aspect of these data are the excessively large number of reports associated with hour 0 (midnight up to but excluding 1 am), which would probably strike us as suspicious. Indeed, the reason is that there are some complaints that are dated but with no associated time, which was recorded in the data as exactly `00:00:00.000`.

In [74]:
query = '''
  SELECT COUNT(*)
    FROM data
    WHERE STRFTIME('%H:%M:%f', CreatedDate) = '00:00:00.000'
'''

pd.read_sql_query(query, disk_engine)

,COUNT(*)
0,532285


What is the most common hour for noise complaints? We compute a tibble called `df_noisy_by_hour` whose variables are {`hour`, `count`} and whose observations are the number of noise complaints that occurred during a given `hour`. Consider a "noise complaint" to be any complaint string containing the word `noise`. We filter out any dates _without_ an associated time, i.e., a timestamp of `00:00:00.000`.

In [75]:
query = '''SELECT strftime('%H', CreatedDate) as hour, count(*) as count
            FROM data
            WHERE (strftime('%H:%M:%f', CreatedDate) <> '00:00:00.000') and 
                  (ComplaintType like '%noise%') 
            GROUP BY hour'''
df_noisy_by_hour = pd.read_sql_query(query, disk_engine)
display(df_noisy_by_hour)

,hour,count
0,00,15349
1,01,11284
2,02,7170
3,03,4241
4,04,3083
5,05,2084
6,06,2832
7,07,3708
8,08,4553
9,09,5122


In [77]:
p = make_barchart(df_noisy_by_hour, 'hour', 'count',
                  {'title': 'Noise complaints by hour',
                   'width': 800, 'height': 320})
show(p, notebook_handle=True)

We create a line chart to show the fraction of complaints (y-axis) associated with each hour of the day (x-axis), with each complaint type shown as a differently colored line. Show just the top 5 complaints (`top_complaints[:5]`). Exclude complaints with a zero-timestamp (i.e., `00:00:00.000`).

In [78]:
# import holoviews as hv
# hv.extension('bokeh')
# from holoviews import Bars

### BEGIN SOLUTION
query1 = '''
  SELECT STRFTIME('%H', CreatedDate) AS hour, LOWER(ComplaintType) AS complaint_type, COUNT(*) AS count
    FROM data
    WHERE CreatedDate NOT LIKE "%00:00:00.000%"
    GROUP BY hour, complaint_type
'''

query2 = '''
  SELECT COUNT(*) AS freq, STRFTIME('%H', CreatedDate) AS hour
    FROM data
    WHERE CreatedDate NOT LIKE "%00:00:00.000%"
    GROUP BY hour
'''

query3 = '''
  SELECT LOWER(ComplaintType) AS complaint_type, COUNT(*) AS num
    FROM data
    GROUP BY complaint_type
    ORDER BY -num
    LIMIT 5
'''

df_query1 = pd.read_sql_query(query1, disk_engine)
df_query2 = pd.read_sql_query(query2, disk_engine)
df_query3 = pd.read_sql_query(query3, disk_engine)

A = df_query1.merge(df_query3, on=['complaint_type'],how='inner')
B = A.merge(df_query2, on=["hour"],how='inner')
print(B)
B = B[['freq','hour','complaint_type','count']]

df_cast = cast(B, key='complaint_type', value='count')

df_new = df_cast.copy()

for i in df_new.columns[2:]:
    df_new[i] = df_new[i]/df_new["freq"]
    
df_top5_frac = df_new.copy()
# del df_top5_frac["freq"]

print(df_top5_frac)

    hour          complaint_type  count     num   freq
0     00        blocked driveway   3030   95080  32418
1     00          heat/hot water    139  241430  32418
2     00         illegal parking   2573   83961  32418
3     00        street condition   1298  124347  32418
4     00  street light condition    891   98577  32418
..   ...                     ...    ...     ...    ...
115   23        blocked driveway   4435   95080  47113
116   23          heat/hot water    243  241430  47113
117   23         illegal parking   3959   83961  47113
118   23        street condition   1838  124347  47113
119   23  street light condition   2045   98577  47113

[120 rows x 5 columns]
complaint_type    freq hour  blocked driveway  heat/hot water  \
0                 8692   04          0.110216        0.005983   
1                10164   03          0.102322        0.004427   
2                10224   05          0.139769        0.005966   
3                15226   02          0.090569        0.0


The provided data presents a summary of the frequency (freq) distribution of the top 5 complaint types ('blocked driveway,' 'heat/hot water,' 'illegal parking,' 'street condition,' 'street light condition') across different hours of the day. The 'hour' column represents the time of the day, and each complaint type's frequency is normalized by the total number of complaints for that hour (count) to calculate the fraction of complaints (num) and the overall frequency (freq).

Key observations:

* Hourly Patterns:

The table reveals hourly patterns for each complaint type, indicating when certain issues are more likely to be reported.
For instance, 'street light condition' complaints tend to peak in the evening hours, while 'illegal parking' complaints show a rise during morning and evening rush hours.

* Top Complaints:

'Blocked driveway' and 'street condition' complaints exhibit a noticeable peak in the early morning hours, potentially related to issues during the night.
'Heat/hot water' complaints are relatively consistent throughout the day, but they experience a slight increase during the morning.

* Crowded City Impact:

The frequency of complaints is significantly higher during the daytime, likely reflecting the increased activity and population density in a crowded city like New York.
Complaints related to 'illegal parking' and 'street condition' show elevated levels during rush hours, suggesting the impact of congestion and traffic-related issues.